In [1]:
import requests
import json
import pandas as pd
import time

In [3]:
def request_data()
content = []
for i in range(60):
    page = f"page[]={i}"
    per_page = f"per_page[]=100"
    url = f"https://www.balldontlie.io/api/v1/games?{page}&{per_page}"
    response = requests.get(url)
    content.append(response.json())
#     time.sleep(0.1)

In [4]:
rows = []
for i in range(len(content)):
    for n in range(len(content[i]["data"])):
        rows.append(content[i]["data"][n])

In [13]:
basketball = pd.DataFrame(rows)
basketball.head()

,date,home_team,home_team_score,id,period,postseason,season,status,time,visitor_team,visitor_team_score
0,2019-01-30T00:00:00.000Z,"{'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...",126,47179,4,False,2018,Final,,"{'id': 4, 'abbreviation': 'CHA', 'city': 'Char...",94
1,2019-02-09T00:00:00.000Z,"{'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...",112,48751,4,False,2018,Final,,"{'id': 13, 'abbreviation': 'LAC', 'city': 'LA'...",123
2,2019-02-08T00:00:00.000Z,"{'id': 23, 'abbreviation': 'PHI', 'city': 'Phi...",117,48739,4,False,2018,Final,,"{'id': 8, 'abbreviation': 'DEN', 'city': 'Denv...",110
3,2019-02-08T00:00:00.000Z,"{'id': 30, 'abbreviation': 'WAS', 'city': 'Was...",119,48740,4,False,2018,Final,,"{'id': 6, 'abbreviation': 'CLE', 'city': 'Clev...",106
4,2019-02-08T00:00:00.000Z,"{'id': 26, 'abbreviation': 'SAC', 'city': 'Sac...",102,48746,4,False,2018,Final,,"{'id': 16, 'abbreviation': 'MIA', 'city': 'Mia...",96


In [14]:
basketball = basketball[["home_team",
                         "home_team_score",
                         "season",
                         "visitor_team",
                         "visitor_team_score"]]

In [15]:
basketball = pd.concat([basketball.drop(["home_team"], axis=1),
                        basketball["home_team"].apply(pd.Series)],
                       axis=1)
basketball.rename(columns={"conference":"home_conference",
                          "full_name":"home_name"}, inplace=True)
basketball = pd.concat([basketball.drop(["visitor_team"], axis=1),
                        basketball["visitor_team"].apply(pd.Series)],
                       axis=1)
basketball.rename(columns={"conference":"visitor_conference",
                          "full_name": "visitor_name"}, inplace=True)

In [16]:
basketball = basketball[["season",
                         "home_conference",
                         "home_name",
                         "home_team_score",
                         "visitor_conference",
                         "visitor_name",
                         "visitor_team_score"]]

In [17]:
basketball = basketball.loc[((basketball["home_conference"] == "East")
                             & (basketball["visitor_conference"] == "West"))
                            | (((basketball["home_conference"] == "West")
                             & (basketball["visitor_conference"] == "East")))]

In [19]:
basketball["win"] = basketball.apply(lambda x: "East" if (
                                                        ((x["home_conference"] == "East")
                                                        & (x["home_team_score"] > x["visitor_team_score"]))
                                                        | ((x["visitor_conference"] == "East")
                                                         & (x["home_team_score"] < x["visitor_team_score"]))
                                                        )
                                     else ("West" if (
                                                    ((x["home_conference"] == "West")
                                                    & (x["home_team_score"] > x["visitor_team_score"]))
                                                    | ((x["visitor_conference"] == "West")
                                                    & (x["home_team_score"] < x["visitor_team_score"]))
                                                    )
                                        else "draw"), 
                                     axis=1)
basketball.head()

,season,home_conference,home_name,home_team_score,visitor_conference,visitor_name,visitor_team_score,win
1,2018,East,Boston Celtics,112,West,LA Clippers,123,West
2,2018,East,Philadelphia 76ers,117,West,Denver Nuggets,110,East
4,2018,West,Sacramento Kings,102,East,Miami Heat,96,West
12,2018,East,Philadelphia 76ers,143,West,Los Angeles Lakers,120,East
16,2018,West,Golden State Warriors,120,East,Miami Heat,118,West


In [20]:
basketball.win.value_counts()

West    180
East    180
Name: win, dtype: int64